In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.1/529.1 kB 9.5 MB/s eta 0:00:00


----------------------------------------------------TEST----------------------------------------------------

In [4]:

import os
from google.cloud import vision

# ✅ Thiết lập credentials đúng file JSON của bạn
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/credentials/projectsummarize-fbeb4ca13ea2.json"

# 🔹 Khởi tạo client
client = vision.ImageAnnotatorClient()

# 🔹 Hàm OCR
def google_vision_ocr(image_path):
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    return response.full_text_annotation.text

# ✅ Test OCR với ảnh của bạn
image_path = "/content/drive/MyDrive/images/Screenshot 2025-10-04 154203.png"  # thay nếu khác
text = google_vision_ocr(image_path)

print("===== Kết quả OCR =====")
print(text)


===== Kết quả OCR =====
Bài Mở đầu
Em là học sinh
Em việt
Em nói
Em doc
Đồ dùng học tập
Đồ dùng học tập
Em làm việc nhóm
Em trải nghiêm
Em học ở nhà


----------------------------------------------------START----------------------------------------------------

In [6]:
import os
from google.cloud import vision

# Đảm bảo Drive đã được mount và credentials đã set
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/credentials/projectsummarize-fbeb4ca13ea2.json"

client = vision.ImageAnnotatorClient()

def google_vision_ocr(image_path):
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    return response.full_text_annotation.text

# 🔹 Thư mục chứa ảnh
image_folder = "/content/drive/MyDrive/images5/"
output_file = "/content/drive/MyDrive/export_to_text/vision_text5.txt"

# 🔹 Mở file để ghi (chế độ 'a' = append thêm nội dung mới)
with open(output_file, "a", encoding="utf-8") as f:
    for filename in os.listdir(image_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(image_folder, filename)
            print(f"🔍 Đang OCR ảnh: {filename}")
            text = google_vision_ocr(image_path)

            # Ghi vào file với tiêu đề ảnh
            f.write(f"\n===== ẢNH: {filename} =====\n")
            f.write(text + "\n")

print("✅ Hoàn thành OCR toàn bộ ảnh và lưu vào vision_text2.txt!")


🔍 Đang OCR ảnh: Screenshot 2025-10-12 161114.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161129.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161136.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161138.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161057.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161149.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161142.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161151.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161154.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161157.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161201.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161207.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161214.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161218.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161221.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161226.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161233.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161239.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161249.png
🔍 Đang OCR ảnh: Screenshot 2025-10-12 161253.png
🔍 Đang OCR ảnh: Scre

KeyboardInterrupt: 

In [7]:
# =====================
# Colab script: Sửa dấu + sửa lỗi nhẹ bằng từ điển (offline)
# =====================

# Yêu cầu: đã mount Drive trước đó

import json
import unicodedata
import re
from collections import defaultdict
from tqdm import tqdm

# ---------------------
# Cấu hình đường dẫn
# ---------------------
DICT_PATH = "/content/drive/MyDrive/vietnamese_dict.txt"   # file JSONL bạn đã có
INPUT_FILE = "/content/drive/MyDrive/export_to_text/vision_text5.txt"
OUTPUT_FILE = "/content/drive/MyDrive/export_to_text/vision_text_fixed5.txt"

# ---------------------
# Hàm tiện ích: remove accents (chuẩn hoá không dấu)
# ---------------------
def remove_accents(s: str) -> str:
    # chuẩn hoá NFD rồi loại bỏ các ký tự combining
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')
    # chuyển một số ký tự đặc biệt về chuẩn ascii
    s = s.replace('đ', 'd').replace('Đ', 'D')
    return unicodedata.normalize('NFC', s)

# ---------------------
# Hàm Levenshtein với ngưỡng dừng sớm (early stop)
# ---------------------
def levenshtein_with_max(a: str, b: str, max_dist: int):
    # returns distance if <= max_dist, else returns a value > max_dist
    # optimized: use two-row DP
    if abs(len(a) - len(b)) > max_dist:
        return max_dist + 1
    if len(a) < len(b):
        a, b = b, a
    previous = list(range(len(b) + 1))
    for i, ca in enumerate(a, start=1):
        current = [i] + [0] * len(b)
        # compute current row with pruning
        min_in_row = current[0]
        for j, cb in enumerate(b, start=1):
            insert_cost = current[j-1] + 1
            delete_cost = previous[j] + 1
            replace_cost = previous[j-1] + (0 if ca == cb else 1)
            current[j] = min(insert_cost, delete_cost, replace_cost)
            if current[j] < min_in_row:
                min_in_row = current[j]
        if min_in_row > max_dist:
            return max_dist + 1
        previous = current
    return previous[-1]

# ---------------------
# Load từ điển JSONL -> tập các từ hợp lệ + map không dấu -> list từ
# ---------------------
vietnamese_words = set()
deaccent_map = defaultdict(list)

print("⏳ Đang load từ điển từ:", DICT_PATH)
with open(DICT_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
            word = item.get("text", "").strip()
            if not word:
                continue
            # chuẩn hoá: chữ thường
            key = word.lower()
            # chỉ giữ nếu có chữ (có thể chứa space cho cụm từ)
            # loại bỏ những dòng có kí tự lạ ở đầu
            if any(ch.isalpha() for ch in key):
                vietnamese_words.add(key)
                deacc = remove_accents(key).lower()
                deaccent_map[deacc].append(key)
        except Exception as e:
            # bỏ qua dòng lỗi
            continue

print("✅ Đã load từ điển. Số từ (cụm từ) trong bộ từ điển:", len(vietnamese_words))
print("Ví dụ 20 từ:", list(vietnamese_words)[:20])

# ---------------------
# Tokenize đơn giản (giữ punctuation để phục hồi)
# ---------------------
token_split_re = re.compile(r"(\w+|[^\w\s])", flags=re.UNICODE)

def smart_tokenize(line: str):
    # trả về list token, giữ nguyên dấu câu là token riêng
    return token_split_re.findall(line)

def is_number_only(s: str):
    s_stripped = s.strip()
    # nếu chỉ có số hoặc chỉ chứa số+punct
    return bool(re.fullmatch(r"[\d\W]+", s_stripped)) and any(ch.isdigit() for ch in s_stripped)

# ---------------------
# Hàm sửa 1 token bằng từ điển
# ---------------------
def correct_token(token: str, max_lev=2):
    """
    token: token nguyên bản (có thể kèm hoa thường)
    trả về token đã sửa (giữ nguyên kiểu hoa)
    logic:
      1. nếu token trong dict -> giữ
      2. thử tìm trong deaccent_map (khớp không dấu)
      3. tìm từ gần nhất bằng levenshtein trong dictionary nhưng chỉ xét từ có độ dài tương đương
    """
    orig = token
    # giữ nguyên nếu token là punctuation
    if not any(ch.isalpha() for ch in token):
        return token

    # tách prefix/suffix punctuation (ví dụ: "Em," -> 'Em' và ',')
    prefix = re.match(r'^[^\w]*', token).group(0) if re.match(r'^[^\w]*', token) else ''
    suffix = re.search(r'[^\w]*$', token).group(0) if re.search(r'[^\w]*$', token) else ''
    core = token[len(prefix): len(token)-len(suffix) if suffix else None]

    if not core:
        return token

    # remember capitalization
    is_title = core.istitle()
    is_upper = core.isupper()

    core_low = core.lower()

    # 1) exact match
    if core_low in vietnamese_words:
        corrected = core_low
    else:
        # 2) match via deaccent map
        de = remove_accents(core_low)
        candidates = deaccent_map.get(de, [])
        if candidates:
            # choose candidate with minimal levenshtein to core_low (with limit)
            best = None
            bestd = max_lev + 10
            for c in candidates:
                d = levenshtein_with_max(core_low, c, max_lev)
                if d <= max_lev and d < bestd:
                    best = c
                    bestd = d
            if best:
                corrected = best
            else:
                # if none within max_lev, choose first candidate as fallback
                corrected = candidates[0]
        else:
            # 3) fallback: scan dictionary for nearby words (expensive but limited by length)
            best = None
            bestd = max_lev + 10
            L = len(core_low)
            # iterate through words of similar length only to reduce cost
            # choose lengths L-1, L, L+1
            for w in vietnamese_words:
                if abs(len(w) - L) > 1:
                    continue
                d = levenshtein_with_max(core_low, w, max_lev)
                if d <= max_lev and d < bestd:
                    best = w
                    bestd = d
                    if bestd == 0:
                        break
            if best:
                corrected = best
            else:
                # không tìm được -> giữ nguyên
                corrected = core_low

    # restore capitalization similar to original
    if is_upper:
        out = corrected.upper()
    elif is_title:
        out = corrected.capitalize()
    else:
        out = corrected

    return prefix + out + suffix

# ---------------------
# Xử lý toàn file
# ---------------------
print("⏳ Bắt đầu xử lý file:", INPUT_FILE)
with open(INPUT_FILE, "r", encoding="utf-8") as fin, \
     open(OUTPUT_FILE, "w", encoding="utf-8") as fout:
    for raw_line in tqdm(fin):
        line = raw_line.rstrip("\n")
        stripped = line.strip()

        # Giữ nguyên các dòng định dạng ảnh
        if stripped.startswith("=====") and "ẢNH" in stripped:
            fout.write(line + "\n")
            continue

        # Bỏ các dòng chỉ toàn số / ký tự rác
        if is_number_only(stripped):
            # bỏ hoàn toàn (hoặc có thể ghi newline nếu muốn giữ vị trí)
            # chúng ta ghi 1 dòng rỗng để giữ đôi chút format
            fout.write("\n")
            continue

        if stripped == "":
            fout.write("\n")
            continue

        # Tokenize và sửa từng token
        tokens = smart_tokenize(line)
        corrected_tokens = [correct_token(t) for t in tokens]
        new_line = "".join(
            # smart join: nếu token là word and next token is word, insert space
            # but tokenization keeps words and punctuation separate; we'll rebuild with spaces when needed
            corrected_tokens[i] + ("" if (i+1 < len(corrected_tokens) and re.match(r'[^\w\s]', corrected_tokens[i+1])) else " ")
            for i in range(len(corrected_tokens))
        ).rstrip()

        fout.write(new_line + "\n")

print("✅ Hoàn tất! File kết quả lưu tại:", OUTPUT_FILE)


⏳ Đang load từ điển từ: /content/drive/MyDrive/vietnamese_dict.txt
✅ Đã load từ điển. Số từ (cụm từ) trong bộ từ điển: 36531
Ví dụ 20 từ: ['nương náu', 'non choẹt', 'tiết lộ', 'chở', 'duy thực', 'chim khuyên', 'đá magma', 'quý vị', 'thọ', 'cứu viện', 'con dấu', 'sôi', 'một lô một lốc', 'ướt rượt', 'quặt', 'dao phay', 'kê kích', 'mốc xì', 'phòng mạch', 'tên lửa đạn đạo']
⏳ Bắt đầu xử lý file: /content/drive/MyDrive/export_to_text/vision_text5.txt


1326it [00:00, 1669.95it/s]

✅ Hoàn tất! File kết quả lưu tại: /content/drive/MyDrive/export_to_text/vision_text_fixed5.txt


In [8]:
import json

dict_path = "/content/drive/MyDrive/vietnamese_dict.txt"  # ✅ Đường dẫn bạn cung cấp

vietnamese_words = set()
with open(dict_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line.strip())
            word = item.get("text", "").strip().lower()
            if word and word.replace(" ", "").isalpha():  # Cho phép cụm từ có dấu cách
                vietnamese_words.add(word)
        except:
            pass

print("✅ Tổng số từ trong từ điển:", len(vietnamese_words))
print(list(vietnamese_words)[:20])  # xem thử

# ==========================================
# 3️⃣ Hàm lọc từng dòng
# ==========================================
def clean_line(line):
    words = line.split()
    cleaned = [w for w in words if w.lower().strip(".,!?;:()\"'") in vietnamese_words]
    return " ".join(cleaned)

# ==========================================
# 4️⃣ Đọc file OCR gốc → tạo file sạch
# ==========================================
input_file = "/content/drive/MyDrive/export_to_text/vision_text_fixed5.txt"
output_file = "/content/drive/MyDrive/export_to_text/vision_text_clean5.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        cleaned = clean_line(line)
        fout.write(cleaned + "\n")

print("✅ Đã lọc xong! File mới tại:", output_file)


✅ Tổng số từ trong từ điển: 36167
['nương náu', 'non choẹt', 'tiết lộ', 'chở', 'duy thực', 'chim khuyên', 'đá magma', 'quý vị', 'thọ', 'cứu viện', 'con dấu', 'sôi', 'một lô một lốc', 'ướt rượt', 'quặt', 'dao phay', 'kê kích', 'mốc xì', 'phòng mạch', 'tên lửa đạn đạo']
✅ Đã lọc xong! File mới tại: /content/drive/MyDrive/export_to_text/vision_text_clean5.txt
